<a href="https://colab.research.google.com/github/GustavoAdolf0/data_contest/blob/main/calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Store Sales - Time Series Forecasting


*   Revisar Data

*   limpiar
*   unir


*   aplicar primer modelo



In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive 

In [ ]:
path = "drive/MyDrive/store_sales/"

In [ ]:
# data del kaggle
holyday_events= pd.read_csv(path+"holidays_events.csv")
add_holy= pd.read_excel(path+"holy_add.xlsx")

In [ ]:
len(holyday_events), len(add_holy)

(350, 24)

In [ ]:
holyday_events = holyday_events.append(add_holy, ignore_index=True)
# agregar la info abajo del original
len(holyday_events)

374

In [ ]:
#hay eventos locales y nacionales, los transferred corresponden a días que se movieron, 
#quizas hay que generar un calendario que además de tener las festividades añada la info de los pagos

holyday_events["locale_name"].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [ ]:
holyday_events["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [ ]:
#hay que leer el archivo de stores para saber si las ubicaciones conversan
stores= pd.read_csv(path+"stores.csv")
stores["state"].unique()

array(['Pichincha', 'Santo Domingo de los Tsachilas', 'Cotopaxi',
       'Chimborazo', 'Imbabura', 'Bolivar', 'Pastaza', 'Tungurahua',
       'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja', 'El Oro',
       'Esmeraldas', 'Manabi'], dtype=object)

In [ ]:
state_stores= sorted(stores.city.unique())
locale=sorted(holyday_events.locale_name.unique())
len(state_stores), len(locale)

(22, 24)

In [ ]:
locale

['Ambato',
 'Cayambe',
 'Cotopaxi',
 'Cuenca',
 'Ecuador',
 'El Carmen',
 'Esmeraldas',
 'Guaranda',
 'Guayaquil',
 'Ibarra',
 'Imbabura',
 'Latacunga',
 'Libertad',
 'Loja',
 'Machala',
 'Manta',
 'Puyo',
 'Quevedo',
 'Quito',
 'Riobamba',
 'Salinas',
 'Santa Elena',
 'Santo Domingo',
 'Santo Domingo de los Tsachilas']

In [ ]:
state_stores

['Ambato',
 'Babahoyo',
 'Cayambe',
 'Cuenca',
 'Daule',
 'El Carmen',
 'Esmeraldas',
 'Guaranda',
 'Guayaquil',
 'Ibarra',
 'Latacunga',
 'Libertad',
 'Loja',
 'Machala',
 'Manta',
 'Playas',
 'Puyo',
 'Quevedo',
 'Quito',
 'Riobamba',
 'Salinas',
 'Santo Domingo']

In [ ]:
no_locale=[]

for i in range(len(locale)):
   if locale[i] not in state_stores:
     no_locale.append(locale[i])
print(no_locale)

['Cotopaxi', 'Ecuador', 'Imbabura', 'Santa Elena', 'Santo Domingo de los Tsachilas']


son 5 ubicaciones que no están en stores, 
['Cotopaxi', 'Ecuador', 'Imbabura', 'Santa Elena', 'Santo Domingo de los Tsachilas']



1.   Ecuador es nacional para todo el territorio
2.   Santo Domingo de los Tsachilas tiene sólo una diferencia de escritura
3.   Santa Elena no hay tiendas
4.   Imbabura    no hay tiendas
5.   Cotopaxi    no hay tiendas




In [ ]:

holyday_events.head(2)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [ ]:
holyday_events.type.unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [ ]:
#dado que hay pocos, day work se eliminan
holyday_events=holyday_events.drop(holyday_events.index[holyday_events["type"]=='Work Day'])

In [ ]:
holyday_events.loc[holyday_events['type'] =="Additional", 'holyday'] = 1
holyday_events.loc[holyday_events['type'] =="Bridge", 'holyday'] = 1
holyday_events.loc[holyday_events['type'] =="Holiday", 'holyday'] = 1
holyday_events.loc[holyday_events['type'] =="Transfer", 'holyday'] = 1

holyday_events["holyday"]=holyday_events["holyday"].fillna(0)

holyday_events.loc[holyday_events['transferred'] == True, 'aux_1'] = -1
holyday_events["aux_1"]=holyday_events["aux_1"].fillna(0)

holyday_events["holyday"] = holyday_events["holyday"] + holyday_events["aux_1"]
#según 4 condiciones se genera una día feriado, para ajustar los días transferidos se genera una col aux con -1, luego se suma holyday elimiando los días feriados transferidos

holyday_events.drop(["aux_1"], axis=1, inplace=True)
# genera una col con 1 y 0 para ahcer la diferencia entre feriados y días no feriados, los transferred True no son feriados dado que fueron movidos
#dado que son sólo 4 días workdays no se toman en cuenta

In [ ]:
holyday_events.isna().sum()
#revisar que no hay nan

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
holyday        0
dtype: int64

In [ ]:
print(holyday_events.dtypes)

date            object
type            object
locale          object
locale_name     object
description     object
transferred       bool
holyday        float64
dtype: object


In [ ]:
holyday_events.loc[holyday_events['type'] =="Event", 'Event'] = 1
holyday_events["Event"]=holyday_events["Event"].fillna(0)
holyday_events.tail(5)

,date,type,locale,locale_name,description,transferred,holyday,Event
369,2014-01-07,Event,National,Ecuador,dia de reyes,False,0.0,1.0
370,2015-01-08,Event,National,Ecuador,dia de reyes,False,0.0,1.0
371,2016-01-09,Event,National,Ecuador,dia de reyes,False,0.0,1.0
372,2017-01-10,Event,National,Ecuador,dia de reyes,False,0.0,1.0
373,2018-01-11,Event,National,Ecuador,dia de reyes,False,0.0,1.0


In [ ]:
#ahora hay que determinar familias de feriados, o feriados largos, para eso la col date se debe transformar a fecha , además se creara un calendario al cual se le agregara la info del df holyday events
holyday_events['date'] = pd.to_datetime(holyday_events['date'])

In [ ]:
print(holyday_events.dtypes)

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
holyday               float64
Event                 float64
dtype: object


hay que usar dos data set, uno de ecuador y uno de la zona en particular, todos estáran asociados a ecuador y a SU zona

Luego cada transacción tendra col asociadas a zona y a ecuador
para zona se debe generar un identificador día-zona, ecuador va para todos por lo que no necesita un identificador adicional

In [ ]:
holy_region=holyday_events.drop(holyday_events.index[holyday_events["locale_name"]=="Ecuador"])
holy_region.head(3)

,date,type,locale,locale_name,description,transferred,holyday,Event
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,1.0,0.0
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,1.0,0.0
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,1.0,0.0


In [ ]:
events_region=holy_region[holy_region['type'] == "Event"].copy(deep=True)
holy_region=holy_region.drop(holy_region.index[holy_region["type"]=="Event"])

events_region.head(3)

,date,type,locale,locale_name,description,transferred,holyday,Event


In [ ]:
holy_region['date'].duplicated().sum()

20

In [ ]:
holy_ecuador=holyday_events[holyday_events['locale_name'] == "Ecuador"]
holy_ecuador.head(3)

,date,type,locale,locale_name,description,transferred,holyday,Event
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,1.0,0.0
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,0.0,0.0
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,1.0,0.0


In [ ]:
events_ecuador=holy_ecuador[holy_ecuador['type'] == "Event"].copy(deep=True)
holy_ecuador=holy_ecuador.drop(holy_ecuador.index[holy_ecuador["type"]=="Event"])

events_ecuador.head(3)

,date,type,locale,locale_name,description,transferred,holyday,Event
55,2013-05-12,Event,National,Ecuador,Dia de la Madre,False,0.0,1.0
103,2014-05-11,Event,National,Ecuador,Dia de la Madre,False,0.0,1.0
106,2014-06-12,Event,National,Ecuador,Inauguracion Mundial de futbol Brasil,False,0.0,1.0


In [ ]:
holy_ecuador['date'].duplicated().sum()

3

In [ ]:
holy_ecuador[holy_ecuador.duplicated(['date'], keep=False)]

,date,type,locale,locale_name,description,transferred,holyday,Event
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False,1.0,0.0
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False,1.0,0.0
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False,1.0,0.0
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False,1.0,0.0
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False,1.0,0.0
157,2014-12-26,Additional,National,Ecuador,Navidad+1,False,1.0,0.0


In [ ]:
holy_ecuador.drop_duplicates(subset="date", keep='first', inplace=True)
holy_ecuador['date'].duplicated().sum()

0

In [ ]:
events_ecuador['date'].duplicated().sum()

2

In [ ]:
events_ecuador[events_ecuador.duplicated(['date'], keep=False)]

,date,type,locale,locale_name,description,transferred,holyday,Event
107,2014-06-15,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Suiza,False,0.0,1.0
244,2016-05-08,Event,National,Ecuador,Terremoto Manabi+22,False,0.0,1.0
245,2016-05-08,Event,National,Ecuador,Dia de la Madre,False,0.0,1.0
351,2014-06-15,Event,National,Ecuador,Dia del padre,False,0.0,1.0


In [ ]:
events_ecuador["description"].unique()

array(['Dia de la Madre', 'Inauguracion Mundial de futbol Brasil',
       'Mundial de futbol Brasil: Ecuador-Suiza',
       'Mundial de futbol Brasil: Ecuador-Honduras',
       'Mundial de futbol Brasil: Ecuador-Francia',
       'Mundial de futbol Brasil: Octavos de Final',
       'Mundial de futbol Brasil: Cuartos de Final',
       'Mundial de futbol Brasil: Semifinales',
       'Mundial de futbol Brasil: Tercer y cuarto lugar',
       'Mundial de futbol Brasil: Final', 'Black Friday', 'Cyber Monday',
       'Terremoto Manabi', 'Terremoto Manabi+1', 'Terremoto Manabi+2',
       'Terremoto Manabi+3', 'Terremoto Manabi+4', 'Terremoto Manabi+5',
       'Terremoto Manabi+6', 'Terremoto Manabi+7', 'Terremoto Manabi+8',
       'Terremoto Manabi+9', 'Terremoto Manabi+10', 'Terremoto Manabi+11',
       'Terremoto Manabi+12', 'Terremoto Manabi+13',
       'Terremoto Manabi+14', 'Terremoto Manabi+15',
       'Terremoto Manabi+16', 'Terremoto Manabi+17',
       'Terremoto Manabi+18', 'Terremoto 

In [ ]:
#dado que faltan eventos se añadiran días del padre, día del niño, san valentin,  y día de los reyes


In [ ]:
len(holyday_events)

369

In [ ]:
#dado que hay días con más de un evento, se eliminara el repetido
events_ecuador.drop_duplicates(subset="date", keep='first', inplace=True)

In [ ]:
holy_region[holy_region.duplicated(['date'], keep='first')]

,date,type,locale,locale_name,description,transferred,holyday,Event
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False,1.0,0.0
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False,1.0,0.0
11,2012-07-03,Holiday,Local,El Carmen,Cantonizacion de El Carmen,False,1.0,0.0
59,2013-06-25,Holiday,Local,Machala,Fundacion de Machala,False,1.0,0.0
60,2013-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False,1.0,0.0
62,2013-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False,1.0,0.0
111,2014-06-25,Holiday,Local,Machala,Fundacion de Machala,False,1.0,0.0
112,2014-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,1.0,0.0
119,2014-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False,1.0,0.0
177,2015-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,1.0,0.0


In [ ]:
holy_region["locale_name"].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ambato', 'Ibarra',
       'Quevedo', 'Santo Domingo de los Tsachilas', 'Santa Elena',
       'Quito', 'Loja', 'Salinas', 'Guayaquil'], dtype=object)

In [ ]:
holy_region['date'] = holy_region['date'].astype(str)
holy_region["d_locale"]=holy_region["date"] +"-"+ holy_region["locale_name"]
holy_region.head(2)

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,1.0,0.0,2012-03-02-Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,1.0,0.0,2012-04-01-Cotopaxi


In [ ]:
holy_region[holy_region.duplicated(['d_locale'], keep='first')]

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False,1.0,0.0,2016-07-24-Guayaquil


In [ ]:
holy_region.loc[holy_region['locale_name'] == "Guayaquil"]

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
64,2013-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2013-07-24-Guayaquil
65,2013-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,False,1.0,0.0,2013-07-25-Guayaquil
127,2014-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2014-07-24-Guayaquil
128,2014-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,False,1.0,0.0,2014-07-25-Guayaquil
182,2015-07-24,Holiday,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2015-07-24-Guayaquil
183,2015-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,False,1.0,0.0,2015-07-25-Guayaquil
264,2016-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2016-07-24-Guayaquil
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False,1.0,0.0,2016-07-24-Guayaquil
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True,0.0,0.0,2016-07-25-Guayaquil
321,2017-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2017-07-24-Guayaquil


In [ ]:
#claramente no tiene sentido mantener ambas fechas
holy_region.drop_duplicates(subset="date", keep='first', inplace=True)

In [ ]:
holy_region[holy_region.duplicated(['d_locale'], keep='first')]

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale


In [ ]:
#eliminar las columnas adicionales
holy_ecuador.drop(["Event"], axis=1, inplace=True)
holy_ecuador.drop(["transferred"], axis=1, inplace=True)
holy_ecuador.drop(["locale"], axis=1, inplace=True)
holy_ecuador.drop(["locale_name"], axis=1, inplace=True)

events_ecuador.drop(["holyday"], axis=1, inplace=True)
events_ecuador.drop(["transferred"], axis=1, inplace=True)
events_ecuador.drop(["locale"], axis=1, inplace=True)
events_ecuador.drop(["locale_name"], axis=1, inplace=True)
events_ecuador.drop(["type"], axis=1, inplace=True)


events_ecuador= events_ecuador.rename(columns={'description': 'desc_events'})
holy_ecuador= holy_ecuador.rename(columns={'description': 'desc_holy'})



ya no hay datos repetidos y tenemos 3 df

holy_region en donde están las festiviades asociadas a región, todavía faltan estandarizar los nombres de las regiones para santo domingo y santo domingo de los tsachilas

events_ecuador posee los eventos como terremotos o día de la madre mundiales etc

holy_ecuador posee los feriados que se comparten para todos los territorios

In [ ]:
import calendar
from pandas.tseries.offsets import MonthEnd

In [ ]:
def create_date_table(start, end):
    start_ts = pd.to_datetime(start).date()

    end_ts = pd.to_datetime(end).date()

    # record timetsamp is empty for now
    dates =  pd.DataFrame(columns=['Record_timestamp'], index=pd.date_range(start_ts, end_ts))
    dates.index.name = 'date'

    days_names = {
        i: name
        for i, name
        in enumerate(['Monday', 'Tuesday', 'Wednesday',
                      'Thursday', 'Friday', 'Saturday', 
                      'Sunday'])
    }

    #dates['DOW'] = dates.index.dayofweek.map(days_names.get)
    dates["DOW2"] = dates.index.dayofweek+1
    dates["Day"] = dates.index.day
    dates['Month'] = dates.index.month
    dates['Quarter'] = dates.index.quarter
    dates['Semester'] = dates.index.month.map(lambda mth: 1 if mth <7 else 2)
    dates['Year'] = dates.index.year  #agregar info de días , mes etc

    dates['DOY'] = dates.index.dayofyear   #día del año
    dates.loc[dates['Day'] == 15, 'pub_pay'] = 1  #pagos días 15

    dates['aux1'] = dates["Day"]-1
    dates["aux1"] = dates["aux1"].shift(periods=-1)
    dates.loc[dates["aux1"] == 0 , "priv_pay"] = 1  #día de fin de mes

    dates.loc[dates["DOW2"] == 6, "FDS"] = 1
    dates.loc[dates["DOW2"] == 7, "FDS"] = 1 #info fin de semana
    dates['FDS'] = dates['FDS'].fillna(0)

    dates['pub_pay'] = dates['pub_pay'].fillna(0)
    dates['priv_pay'] = dates['priv_pay'].fillna(0)

    dates['pub_pay'] = dates['pub_pay'] + dates['priv_pay']

    dates.reset_index(inplace=True)
    dates.index.name = 'date_id'
    dates.drop(["Record_timestamp"], axis=1, inplace=True)
    dates.drop(["aux1"], axis=1, inplace=True)

    dates["DOW2"] = dates["DOW2"].astype(int)
    dates["Day"] = dates["Day"].astype(int)
    dates["Month"] = dates["Month"].astype(int)
    dates["Quarter"] = dates["Quarter"].astype(int)
    dates["Semester"] = dates["Semester"].astype(int)
    dates["Year"] = dates["Year"].astype(int)
    dates["DOY"] = dates["DOY"].astype(int)
    dates["pub_pay"] = dates["pub_pay"].astype(int)
    dates["priv_pay"] = dates["priv_pay"].astype(int)
    dates["FDS"] = dates["FDS"].astype(int)

    return dates

In [ ]:
c_train = create_date_table(start='2013-01-01', end='2017-08-15')
c_test = create_date_table(start='2017-08-16', end='2017-08-31')


In [ ]:
print(c_test.dtypes)

date        datetime64[ns]
DOW2                 int64
Day                  int64
Month                int64
Quarter              int64
Semester             int64
Year                 int64
DOY                  int64
pub_pay              int64
priv_pay             int64
FDS                  int64
dtype: object


In [ ]:
print(holy_ecuador.dtypes)

date         datetime64[ns]
type                 object
desc_holy            object
holyday             float64
dtype: object


In [ ]:
print(events_ecuador.dtypes)

date           datetime64[ns]
desc_events            object
Event                 float64
dtype: object


In [ ]:
holy_ecuador['date'].duplicated().sum(), events_ecuador['date'].duplicated().sum(), c_test['date'].duplicated().sum(), c_train['date'].duplicated().sum()

(0, 0, 0, 0)

In [ ]:
#unir los datos para presentar

c_test=pd.merge(c_test, holy_ecuador, on="date", how="left")
c_test=pd.merge(c_test, events_ecuador, on="date", how="left")

c_train=pd.merge(c_train, holy_ecuador, on="date", how="left")
c_train=pd.merge(c_train, events_ecuador, on="date", how="left")

In [ ]:
c_test = c_test.fillna(0)
c_train = c_train.fillna(0)

In [ ]:
c_test

,date,DOW2,Day,Month,Quarter,Semester,Year,DOY,pub_pay,priv_pay,FDS,type,desc_holy,holyday,desc_events,Event
0,2017-08-16,3,16,8,3,2,2017,228,0,0,0,0,0,0.0,0,0.0
1,2017-08-17,4,17,8,3,2,2017,229,0,0,0,0,0,0.0,0,0.0
2,2017-08-18,5,18,8,3,2,2017,230,0,0,0,0,0,0.0,0,0.0
3,2017-08-19,6,19,8,3,2,2017,231,0,0,1,0,0,0.0,0,0.0
4,2017-08-20,7,20,8,3,2,2017,232,0,0,1,0,0,0.0,0,0.0
5,2017-08-21,1,21,8,3,2,2017,233,0,0,0,0,0,0.0,0,0.0
6,2017-08-22,2,22,8,3,2,2017,234,0,0,0,0,0,0.0,0,0.0
7,2017-08-23,3,23,8,3,2,2017,235,0,0,0,0,0,0.0,0,0.0
8,2017-08-24,4,24,8,3,2,2017,236,0,0,0,0,0,0.0,0,0.0
9,2017-08-25,5,25,8,3,2,2017,237,0,0,0,0,0,0.0,0,0.0


In [ ]:
#los fines de semana largo pueden generar cambios en la demanda
c_test["feriado_fds"]=c_test["holyday"] + c_test["FDS"]
#dado que un feriado puede caer en fin de semana el maximo de esa col debe ser uno
c_test.loc[c_test['feriado_fds'] > 1, 'feriado_fds'] = 1


c_train["feriado_fds"]=c_train["holyday"] + c_train["FDS"]

c_train.loc[c_train['feriado_fds'] > 1, 'feriado_fds'] = 1



calendar_ecuador["aux1"] = calendar_ecuador["feriado_fds"].shift(periods=-1)
calendar_ecuador["aux2"] = calendar_ecuador["feriado_fds"].shift(periods=-2)
calendar_ecuador["aux3"] = calendar_ecuador["feriado_fds"].shift(periods=-3)
calendar_ecuador["aux4"] = calendar_ecuador["feriado_fds"].shift(periods=-4)
calendar_ecuador["aux5"] = calendar_ecuador["feriado_fds"].shift(periods=-5)
calendar_ecuador["aux6"] = calendar_ecuador["feriado_fds"].shift(periods=-6)
# traigo el valor del día de ayer y anteayer..., luego los sumo y si dan 4  son fin de semana largo de 4 días

calendar_ecuador["FDL4"] = calendar_ecuador["aux6"] + calendar_ecuador["aux5"] + calendar_ecuador["aux4"] + calendar_ecuador["aux3"] + calendar_ecuador["aux2"] + calendar_ecuador["aux1"] + calendar_ecuador["feriado_fds"]
calendar_ecuador["FDL4"] = calendar_ecuador["FDL4"].shift(3)
calendar_ecuador.loc[calendar_ecuador['FDL4'] <4, "FDL4"] = 0
calendar_ecuador.loc[calendar_ecuador['FDL4'] >4, "FDL4"] = 0

calendar_ecuador["FDL4"] = calendar_ecuador["FDL4"]*calendar_ecuador["feriado_fds"]



# traigo el valor del día de ayer y anteayer..., luego los sumo y si dan 3 son fin de semana largo de 3 días

calendar_ecuador["FDL3"] = calendar_ecuador["aux4"] + calendar_ecuador["aux3"] + calendar_ecuador["aux2"] + calendar_ecuador["aux1"] + calendar_ecuador["feriado_fds"]
calendar_ecuador["FDL3"] = calendar_ecuador["FDL3"].shift(2)
calendar_ecuador.loc[calendar_ecuador['FDL3'] <3, "FDL3"] = 0
calendar_ecuador.loc[calendar_ecuador['FDL3'] >3, "FDL3"] = 0
calendar_ecuador["FDL3"] = calendar_ecuador["FDL3"]*calendar_ecuador["feriado_fds"]

calendar_ecuador.drop(["aux1"], axis=1, inplace=True)
calendar_ecuador.drop(["aux2"], axis=1, inplace=True)
calendar_ecuador.drop(["aux3"], axis=1, inplace=True)
calendar_ecuador.drop(["aux4"], axis=1, inplace=True)
calendar_ecuador.drop(["aux5"], axis=1, inplace=True)
calendar_ecuador.drop(["aux6"], axis=1, inplace=True)

calendar_ecuador = calendar_ecuador.fillna(0)
calendar_ecuador.head(50)

In [ ]:
c_test["holyday"] = c_test["holyday"].astype(int)
c_test["Event"] = c_test["Event"].astype(int)

c_train["holyday"] = c_train["holyday"].astype(int)
c_train["Event"] = c_train["Event"].astype(int)

c_test["feriado_fds"] = c_test["feriado_fds"].astype(int)
c_train["feriado_fds"] = c_train["feriado_fds"].astype(int)


c_test.head(2)


,date,DOW2,Day,Month,Quarter,Semester,Year,DOY,pub_pay,priv_pay,FDS,type,desc_holy,holyday,desc_events,Event,feriado_fds
0,2017-08-16,3,16,8,3,2,2017,228,0,0,0,0,0,0,0,0,0
1,2017-08-17,4,17,8,3,2,2017,229,0,0,0,0,0,0,0,0,0


In [ ]:
#se añadira a info del petroleo
oil= pd.read_csv(path+"oil.csv")

In [ ]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [ ]:
print(oil.dtypes)

date           object
dcoilwtico    float64
dtype: object


In [ ]:
oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [ ]:
oil['date'].duplicated().sum()

0

In [ ]:
#los nan se pueden llenar usando el valor anterior cuando se unan a calendar_ecuador, la fecha se debe cambiar a fecha
oil['date'] = pd.to_datetime(oil['date'])
oil.tail(5)

,date,dcoilwtico
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [ ]:
c_test=pd.merge(c_test, oil, on="date", how="left")
c_train=pd.merge(c_train, oil, on="date", how="left")

In [ ]:
c_train.fillna(method="ffill", inplace=True)
c_test.fillna(method="ffill", inplace=True)

c_test.fillna(0, inplace=True)
c_train.fillna(0, inplace=True)

c_test.drop(["desc_events","desc_holy","type"], axis=1, inplace=True)
c_train.drop(["desc_events","desc_holy", "type"], axis=1, inplace=True)

c_test.head()



,date,DOW2,Day,Month,Quarter,Semester,Year,DOY,pub_pay,priv_pay,FDS,holyday,Event,feriado_fds,dcoilwtico
0,2017-08-16,3,16,8,3,2,2017,228,0,0,0,0,0,0,46.80
1,2017-08-17,4,17,8,3,2,2017,229,0,0,0,0,0,0,47.07
2,2017-08-18,5,18,8,3,2,2017,230,0,0,0,0,0,0,48.59
3,2017-08-19,6,19,8,3,2,2017,231,0,0,1,0,0,1,48.59
4,2017-08-20,7,20,8,3,2,2017,232,0,0,1,0,0,1,48.59


In [ ]:
c_train.tail()

,date,DOW2,Day,Month,Quarter,Semester,Year,DOY,pub_pay,priv_pay,FDS,holyday,Event,feriado_fds,dcoilwtico
1683,2017-08-11,5,11,8,3,2,2017,223,0,0,0,1,0,1,48.81
1684,2017-08-12,6,12,8,3,2,2017,224,0,0,1,0,0,1,48.81
1685,2017-08-13,7,13,8,3,2,2017,225,0,0,1,0,0,1,48.81
1686,2017-08-14,1,14,8,3,2,2017,226,0,0,0,0,0,0,47.59
1687,2017-08-15,2,15,8,3,2,2017,227,1,0,0,0,0,0,47.57


In [ ]:
c_test.to_csv('drive/MyDrive/store_sales/c_test.csv')
c_train.to_csv('drive/MyDrive/store_sales/c_train.csv')

In [ ]:
holy_region

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,1.0,0.0,2012-03-02-Manta
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,1.0,0.0,2012-04-01-Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,1.0,0.0,2012-04-12-Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,1.0,0.0,2012-04-14-Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,1.0,0.0,2012-04-21-Riobamba
...,...,...,...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False,1.0,0.0,2017-11-12-Ambato
339,2017-12-05,Additional,Local,Quito,Fundacion de Quito-1,False,1.0,0.0,2017-12-05-Quito
340,2017-12-06,Holiday,Local,Quito,Fundacion de Quito,True,0.0,0.0,2017-12-06-Quito
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False,1.0,0.0,2017-12-08-Loja


In [ ]:
start_date = "2013-01-01"
end_date = "2017-08-15"

after_start_date = holy_region["date"] >= start_date
before_end_date = holy_region["date"] <= end_date
between_two_dates = after_start_date & before_end_date
r_train = holy_region.loc[between_two_dates]
r_train



,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
46,2013-03-02,Holiday,Local,Manta,Fundacion de Manta,False,1.0,0.0,2013-03-02-Manta
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,1.0,0.0,2013-04-01-Cotopaxi
48,2013-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,1.0,0.0,2013-04-12-Cuenca
49,2013-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,1.0,0.0,2013-04-14-Libertad
50,2013-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,1.0,0.0,2013-04-21-Riobamba
...,...,...,...,...,...,...,...,...,...
320,2017-07-23,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1.0,0.0,2017-07-23-Cayambe
321,2017-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False,1.0,0.0,2017-07-24-Guayaquil
322,2017-07-25,Additional,Local,Guayaquil,Fundacion de Guayaquil,False,1.0,0.0,2017-07-25-Guayaquil
323,2017-08-05,Holiday,Local,Esmeraldas,Fundacion de Esmeraldas,False,1.0,0.0,2017-08-05-Esmeraldas


In [ ]:
start_date = "2017-08-16"
end_date = "2017-08-31"

after_start_date = holy_region["date"] >= start_date
before_end_date = holy_region["date"] <= end_date
between_two_dates = after_start_date & before_end_date
r_test = holy_region.loc[between_two_dates]
r_test

,date,type,locale,locale_name,description,transferred,holyday,Event,d_locale
327,2017-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,1.0,0.0,2017-08-24-Ambato


In [ ]:

r_train.drop(["Event", "description"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
r_train.groupby(["transferred"]).count()

,date,type,locale,locale_name,holyday,d_locale
transferred,,,,,,
False,117,117,117,117,117,117
True,2,2,2,2,2,2


In [ ]:
#transferred ya no tiene información
r_train.drop(["transferred"], axis=1, inplace=True)
r_test.drop(["transferred"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
r_test.drop(["Event", "description"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
r_test.drop(["type", "locale"], axis=1, inplace=True)
r_train.drop(["type", "locale"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
r_test.to_csv('drive/MyDrive/store_sales/r_test.csv', index=False)
r_train.to_csv('drive/MyDrive/store_sales/r_test.csv', index=False)